In [1]:
import os
import sys

# PyTorch 라이브러리 경로 추가 (확실히 하기 위해 Python 코드에서도 추가)
# os.environ["DYLD_LIBRARY_PATH"] = "/opt/anaconda3/envs/AiGO/lib/python3.12/site-packages/torch/lib:" + os.environ.get("DYLD_LIBRARY_PATH", "")

# 빌드된 .so 파일 경로 추가
sys.path.append('/Users/ijimin/Documents/GitHub/AiGO/Omok/Jimin/MCTS/build/lib.macosx-11.0-arm64-cpython-312')

# 모듈 import
import MCTS_cpp

print("MCTS_cpp module imported successfully!")

MCTS_cpp module imported successfully!


In [3]:
import MCTS_cpp
import torch



# ✅ 더미 PyTorch 모델 정의
class DummyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(64, 10)  # 임의의 선형 레이어 추가

    def forward(self, x):
        # 정책 (확률 분포) 및 가치 반환
        policy = torch.rand(10)  # 더미 정책 분포
        value = torch.tensor(0.5)  # 더미 가치
        return policy, value


# ✅ JIT 모델 변환 및 저장
dummy_model = DummyModel()
model_path = "dummy_model.pt"
scripted_model = torch.jit.script(dummy_model)
scripted_model.save(model_path)

# ✅ JIT 모델 로드
model = torch.jit.load(model_path)

# ✅ MCTS 노드 생성
state = DummyState()
node = MCTS_cpp.Node(state, 1.0)

print(f"✅ Node 초기화 완료: Prior Probability = {node.p}")
print(f"✅ Node 방문 횟수(n) = {node.n}, 누적 가치(w) = {node.w}")

✅ Node 초기화 완료: Prior Probability = 1.0
✅ Node 방문 횟수(n) = 0, 누적 가치(w) = 0.0


In [3]:
node.state()

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]])

In [4]:
# ✅ 더미 상태 클래스 정의
class DummyState:
    def __init__(self):
        self.legal_actions = [0, 1, 2, 3]  # 가능한 액션 4개

    def __call__(self):
        """ 상태를 NumPy 배열 또는 PyTorch 텐서로 반환 """
        return torch.zeros((1, 8, 8))  # 상태 텐서 (더미 데이터)

    def is_done(self):
        """ 현재 게임이 종료되었는지 확인 """
        return False  # 항상 False 반환

    def is_lose(self):
        """ 현재 상태에서 패배 여부 확인 """
        return False  # 항상 False 반환

    def next(self, action):
        """ 다음 상태 반환 (액션 수행 후 상태 변화) """
        return DummyState()

# ✅ PyTorch 모델 정의
class DummyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(8, 2)

    def forward(self, x):
        policy =  self.fc(x)  # 더미 정책 분포
        value = torch.tensor(0.5)  # 더미 가치
        return policy, value

# ✅ 모델 생성 및 JIT 변환
dummy_model = DummyModel()
scripted_model = torch.jit.script(dummy_model)  # JIT 변환
scripted_model.save("dummy_model.pt")           # JIT 모델 저장

# ✅ JIT 모델 로드
model = torch.jit.load("dummy_model.pt")        # JIT 모델 로드

# ✅ 테스트 State 객체 생성
state = DummyState()


In [7]:
# ✅ PyTorch 모델 정의
class JITModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(64, 10)

    def forward(self, x):
        policy = torch.rand(10)  # 더미 정책 분포
        value = torch.tensor(0.5)  # 더미 가치
        return policy, value

# ✅ JIT 변환된 모델 생성
model = JITModel()
print(type(model))
scripted_model = torch.jit.script(model)  # 🔥 JIT 변환 (C++과 호환)
print(type(scripted_model))
scripted_model.save("jit_model.pt")       # JIT 모델 저장
jit_model = torch.jit.load("jit_model.pt")  # ✅ C++과 호환되는 모델 로드
print(type(scripted_model._c))

# ✅ MCTS Node 생성
state = DummyState()
node = MCTS_cpp.Node(state, 1.0)

# ✅ JIT 모델을 사용하여 `evaluate_value()` 호출
value = node.evaluate_value(scripted_model._c)  # 🔥 JIT 변환된 모델 사용
print(f"Evaluated Value: {value}")

<class '__main__.JITModel'>
<class 'torch.jit._script.RecursiveScriptModule'>
<class 'torch.ScriptModule'>
Evaluated Value: 0.5


In [8]:
state = DummyState()
legal_policy, value = MCTS_cpp.predict(jit_model._c, state)

print(f"Legal Policy: {legal_policy}, Value: {value}")

Legal Policy: [0.41550931334495544, 0.2758353054523468, 0.2435254156589508, 0.0651300698518753], Value: 0.5


In [21]:
import torch
import MCTS_cpp

# ✅ PyTorch 모델 정의
class JITModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(64, 10)

    def forward(self, x):
        policy = torch.rand(10)  # 더미 정책 분포
        value = torch.tensor(0.5)  # 더미 가치
        return policy, value

# ✅ JIT 변환
model = JITModel()
scripted_model = torch.jit.script(model)  # 🔥 JIT 변환 (torch::jit::script::Module과 동일)

# ✅ JIT 모델 저장 및 로드
scripted_model.save("jit_model.pt")
jit_model = torch.jit.load("jit_model.pt")  # 🔥 C++과 호환되는 `torch::jit::script::Module` 형태

print(f"JIT 모델 타입: {type(jit_model)}")  # 🔹 <class 'torch.jit._script.RecursiveScriptModule'>

# ✅ 더미 상태 정의
class DummyState:
    def __init__(self):
        self.legal_actions = [0, 1, 2, 3]

    def __call__(self):
        return torch.zeros((1, 8, 8))

# ✅ MCTS 예측 테스트
state = DummyState()
legal_policy, value = MCTS_cpp.predict(jit_model.copy(), state)

print(f"Legal Policy: {legal_policy}, Value: {value}")

JIT 모델 타입: <class 'torch.jit._script.RecursiveScriptModule'>


AttributeError: 'RecursiveScriptModule' object has no attribute 'copy'

In [8]:
# ✅ 모델 예측 테스트
legal_policy, value = MCTS_cpp.predict(model, state)
print(f"✅ 모델 예측 결과: Legal Policy = {legal_policy}, Value = {value}")

# ✅ 평가 함수 테스트
eval_value = node.evaluate_value(model)
print(f"✅ Node 평가 값: {eval_value}")

# ✅ 확장 테스트
node.expand(model)
print(f"✅ Node 확장 완료: 자식 노드 수 = {len(node.child_nodes)}")

# ✅ 다음 자식 노드 선택
if len(node.child_nodes) > 0:
    next_node = node.select_next_child_node()
    print(f"✅ 선택된 자식 노드 Prior Probability = {next_node.p}")

print("🎉 모든 테스트 완료!")

TypeError: predict(): incompatible function arguments. The following argument types are supported:
    1. (arg0: torch::jit::Module, arg1: object) -> tuple[list[float], float]

Invoked with: RecursiveScriptModule(
  original_name=DummyModel
  (fc): RecursiveScriptModule(original_name=Linear)
), <__main__.DummyState object at 0x15d975040>